# W5 Practicals - Performance

### Aims:
* To gain some practical experience in evaluating supervised machine learning
models.
* To produce some assessable work for this subject.

### Procedure:
In Prac W2 we applied k-NN and decision tree models to simple classification and regression datasets. The evaluation of the models was based on a single 70/30 split of the data into training and test data. In this Prac we will look more closely at the evaluation of these models.
> Select one of the questions (3 – 6) from Prac W2 and revisit it for this prac.

> On blackboard you will find a [link](https://docs.google.com/spreadsheets/d/1HAIDBp9ofIeEp5_braHBnwJ-heN8qdaxn5qk3q1c0vo/edit?usp=sharing) to a Google spreadsheet. Go there and enter your answers for Q2 – Q5.

In [ ]:
# Common Imports
import pandas as pd

### Q1: Repeat Q2 from Prac W2 10 times, saving the 10 resulting training and test sets.

In [ ]:
# Specific Imports
from sklearn.model_selection import train_test_split

In [ ]:
CSV_FILE_1 = 'w3classif.csv'
df1 = pd.read_csv(CSV_FILE_1, header=None)
df1.columns = ["Feature1", "Feature2", "Label"]
Xc, yc = df1.iloc[:, :-1].values, df1.iloc[:, -1].values
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.3, random_state=42)


CSV_FILE_2 = 'w3regr.csv'
df2 = pd.read_csv(CSV_FILE_2, header=None)
df2.columns = ["Feature1", "Feature2"]
Xr, yr = df2.iloc[:, :-1].values, df2.iloc[:, -1].values
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.3, random_state=42)

### Q2: Calculate the training and test set errors over all of the datasets from Q1 and calculate the average training and test errors over the 10 trials. Are the averages lower or higher than the values you found in Prac W3 (or alternatively compare with the values for the first of your 10 runs)?


### Question 3: Repeat Q1 and Q2 but use a different split – try 50/50 or 90/10. Compare your average error values with those you found in Q2.

### Q4: Calculate the sample standard deviation of your training and test set error values over the 10 trials from Q2 and Q3. What do you observe?

### Q5: Perform 10-fold cross validation using your model and the (original) dataset (use existing Matlab or python functions to do this). What are the mean and standard devations of the cross-validation error?